# Mixtures of Gaussian Processes with GPclust


This notebook accompanies the paper

**Nonparameteric Clustering of Structured Time Series**  
_James Hensman, Magnus Rattray and Neil D. Lawrence_  
IEEE TPAMI 2014

The code is available at <https://github.com/mathDR/gpclust> . The GPclust module depends on [GPflow](https://github.com/GPflow).  

The hierachical Gaussian process model was fleshed out in 

**Hierarchical Bayesian modelling of gene expression time series  
across irregularly sampled replicates and clusters**  
_James Hensman, Neil D. Lawrence and Magnus Rattray_

http://www.biomedcentral.com/1471-2105/14/252



In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'#'svg' would be better, but eats memory for these big plots.
from matplotlib import pyplot as plt
plt.style.use('ggplot')
import numpy as np
import random
import GPclust
import GPflow

## A simple point process dataset

Here's a simulated dataset that contains the simple features that we expect to have in real data sets: . 

In [2]:
#generate a data set. Here we generate Poisson process counts with a different rate mean for each cluster
Nclust = 10
Nobs = [np.random.randint(20,31) for i in range(Nclust)] #a random number of realisations in each cluster
Nx = 156 # number of time steps for each realization ( 3 years if weekly data)

#random rate function for each cluster (between 0.5 and 5.5)
means = (5.0*np.random.rand(Nclust)+.5)

X = []
Y = []
for n in xrange(Nclust):
    for j in xrange(Nobs[n]):
        Y.append(np.random.poisson(means[n],Nx)[:,None])
        X.append(np.asarray(range(Nx))[:,None])


In [3]:
print (X[0]).shape

(156, 1)


In the plot below, we show the underlying function for each cluster as a smooth red function, and the data associated with the cluster as thinly connected blue crosses. 

In [4]:
if False:
    bins = range(10)
    denom = [np.math.factorial(b) for b in bins]
    count = 0
    for n in range(Nclust):
        plt.subplot(2,Nclust/2,n+1)
        for j in range(Nobs[n]):
            plt.hist(Y[count], bins, alpha=1.0/Nobs[n], color = 'red', normed=True, align='left')
            count = count + 1
        plt.plot(bins,np.power(means[n],bins)*np.exp(-means[n])/denom,'b')
    plt.show()

Constructing and optimizing a model
---

Now that we have generated a data set, it's straightforward to build and optimize a clustering model. First, we need to build two GPy kernels (covariance functions), which will be used to model the underlying function and the replication noise, respecively. We'll take a wild stab at the parameters of these covariances, and let the model optimize them for us later. 

The two kernels model the *underlying* function of the cluster, and the deviations of each gene from that underlying function. If we believe that the only corruption of the data from the cluster mean is i.i.d. noise, we can specify a `GPy.kern.White` covariance. In practice, it's helpful to allow correlated noise. The model of any cluster of genes then has a hierarchical structure, with the unknown cluster-specific mean drawn from a GP, and then each gene in that cluster being drawn from a GP with said unknown mean function. 

To optimize the model with the default optimization settings, we call `m.optimize()`. To invoke the recommended merge-split procedure, call `m.systematic_splits()`. Note that during the splitting procedure, many calls are made to the optimize function. 

In [5]:
Z = np.linspace(np.min(X), np.max(X),10)[:,None]

In [11]:
k_underlying = GPflow.kernels.RBF(input_dim=X[0].shape[0], variance=0.1, lengthscales=0.1)
#k_corruption = GPflow.kernels.RBF(input_dim=1, variance=0.01, lengthscales=0.1) + GPflow.kernels.White(1, variance=0.001)
likelihood = GPflow.likelihoods.Poisson()

m = GPclust.MOGP(X, Y, Z, k_underlying, likelihood, num_clusters=10, alpha=1.0, prior_Z='symmetric')
#m.optimize()
m.systematic_splits(verbose=False)

In [12]:
m.log_likelihood()

-156464.20536226252